# LINE STOCK Agent

In [ ]:
!pip install openai
!pip install yfinance==0.2.38
!pip install langchain
!pip install langchain-groq langchain-huggingface
!pip install langchain-community
!pip install faiss-gpu-cu12
#!pip install faiss-cpu
!pip install tensorflow

#line-bot
!pip install line-bot-sdk
!pip install pyngrok

!pip uninstall ploty
!pip install plotly==5.3.1
!pip install kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.51
    Uninstalling yfinance-0.2.51:
      Successfully uninstalled yfinance-0.2.51
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.0/768.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 47.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/

In [ ]:
import pandas as pd
import yfinance as yf
import datetime as dt
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.output_parsers import RegexParser
from langchain_core.runnables import RunnableSequence
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import re
import os
from langchain.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.agents import tool

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [ ]:
# 設定基本的 LLM
model_name = "llama-3.3-70b-versatile"

llm = ChatGroq(temperature=0, model_name=model_name, groq_api_key=GROQ_API_KEY)
#llm_openai = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### 爬取股號、股名對照表

In [ ]:
# 取得全部股票的股號、股名
def stock_name():
  """
  Fetches stock names and IDs from the Taiwan Stock Exchange website and saves it into csv.

  Returns:
      str: The path to the saved CSV file.
  """
  response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[4].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
  df['股號'] = df['股號'].astype(str)

  # 存成 CSV 檔案
  stock_name_table_path = 'stock_name_table.csv'
  df.to_csv(stock_name_table_path, index=False, encoding='utf-8-sig')

  return stock_name_table_path

In [ ]:
stock_name_table_path = stock_name()

# Agent Tools

### 查詢股名或股號

In [ ]:
# 查詢股名或股號
def query_stock(query):
    """
    Queries stock information (name or ID) from a CSV file.

    Args:
        query (str): The stock name or ID to search for.

    Returns:
        dict: A dictionary containing the stock name and ID, or an error message if not found.
              Example: {'股名': '台積電', '股號': '2330.TW'}
              or {'error': '查無資料'}
    """
    # 載入 CSV 檔案
    df = pd.read_csv('stock_name_table.csv', dtype={'股號': str})

    # 查詢條件
    result = df[(df['股名'] == query) | (df['股號'] == query)]

    # 格式化輸出
    if not result.empty:
        stock_info = result.iloc[0]  # 只取第一筆符合的結果
        return {"股名": stock_info['股名'], "股號": f"{stock_info['股號']}.TW"}
    else:
        return {"error": "查無資料"}

In [ ]:
query_stock(query = '2330')

{'股名': '台積電', '股號': '2330.TW'}

In [ ]:
def stock_price(input_dict):
    stock_id, days = input_dict.split(",")
    days = int(days)

### 取得股價表

In [ ]:
def stock_price(input_dict) -> dict:
    """
    Fetches historical stock price data for a given stock ID and number of days, and saves it as a CSV file.

    After fetching the data, you can use 'extract_stock_price' tool to get stock price information.

    Args:
        input_dict (dict): A dictionary containing 'stock_id' (str) and 'days' (int).
                            Example: {'stock_id': '2330.TW', 'days': 30}

    Returns:
        dict: A dictionary containing the 'stock_price_path' (str) to the saved CSV file.
            Example: {'stock_price_path': 'stock_price/2330.TW.csv'}
            or str if an error occurs.
    """
    stock_id, days = input_dict.split(",")
    days = int(days)

    if stock_id == "大盤":
      stock_id="^TWII"

    end = dt.date.today() # 資料結束時間
    start = end - dt.timedelta(days=days) # 資料開始時間
    # 下載資料
    try:
        df = yf.download(stock_id, start=start, progress=False)

    except:
        return f"無法獲取 {stock_id} 的資訊"

    #更換列名
    if 'Adj Close' in df.columns:
      df.columns = ['調整後收盤價', '收盤價', '最高價', '最低價', '開盤價', '成交量']
    else:
      df.columns = ['收盤價', '最高價', '最低價', '開盤價', '成交量']


    dir_name = "stock_price"
    if not os.path.exists(dir_name):
      os.makedirs(dir_name)

    # 存成 CSV 檔案
    stock_price_path = f'{dir_name}/{stock_id}.csv'
    df.to_csv(stock_price_path, index=True, encoding='utf-8-sig')

    return {"stock_price_path" : f"{stock_price_path}"}

In [ ]:
stock_price("2330.TW,10")

{'stock_price_path': 'stock_price/2330.TW.csv'}

In [ ]:
df = pd.read_csv("stock_price/2330.TW.csv")
df

,Date,調整後收盤價,收盤價,最高價,最低價,開盤價,成交量
0,2025-01-07,1150.0,1160.0,1130.0,1130.0,1130.0,48758688
1,2025-01-08,1110.0,1130.0,1105.0,1105.0,1105.0,45211419
2,2025-01-09,1100.0,1115.0,1100.0,1100.0,1100.0,27263215
3,2025-01-10,1100.0,1110.0,1100.0,1100.0,1100.0,22021522
4,2025-01-13,1110.0,1115.0,1070.0,1075.0,1075.0,56690300
5,2025-01-14,1085.0,1090.0,1075.0,1090.0,1090.0,28695247
6,2025-01-15,1085.0,1085.0,1065.0,1065.0,1065.0,35489482
7,2025-01-16,1095.0,1115.0,1090.0,1105.0,1105.0,47542962
8,2025-01-17,1125.0,1125.0,1095.0,1120.0,1120.0,60289321


###取得股價資料

In [ ]:
def extract_stock_price(stock_price_path):
    """
    Extracts specific stock price information from a CSV file.

    Args:
        stock_price_path (str): The path to the CSV file containing stock price data.
                              This should be a valid file path to the CSV generated by `stock_price` tool.

    Returns:
        dict or str: If successful, returns a dictionary containing '日期', '收盤價', '開盤價', '最高價', '最低價', '成交量',
                      '每日報酬', '漲跌價差' and other column data. If an error occurs, returns a str error message.
                      Example:
                      {
                        '日期': ['2024-01-01', '2024-01-02'],
                        '收盤價': [100.0, 102.0],
                        '開盤價': [99.0, 101.0],
                        '最高價': [101.0, 103.0],
                        '最低價': [98.0, 100.0],
                        '成交量': [1000, 1200],
                        '每日報酬': [nan, 0.02],
                      }

    """

    try:
        df = pd.read_csv(stock_price_path, parse_dates=['Date'])
        df.set_index('Date', inplace=True) # 確保日期欄位是索引

        # 只選取最後 20 筆資料
        df = df.tail(20)

        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(),
            '收盤價': df['收盤價'].tolist(),
            '開盤價': df['開盤價'].tolist(),
            '最高價': df['最高價'].tolist(),
            '最低價': df['最低價'].tolist(),
            '成交量': df['成交量'].tolist(),
            '每日報酬': df['收盤價'].pct_change().tolist(),
        }

        # 動態添加其餘欄位
        for column in df.columns:
            if column not in data:  # 避免重複添加
                data[column] = df[column].tolist()

        # 處理空值
        for key in data:
            data[key] = [np.nan if x is None else x for x in data[key]]

        return data

    except Exception as e:
        return f"[Error] An error occurred: {str(e)}"

In [ ]:
a = extract_stock_price('stock_price/2330.TW.csv')
print(json.dumps(a, ensure_ascii=False, separators=(', ', ': ')))

{"日期": ["2025-01-07", "2025-01-08", "2025-01-09", "2025-01-10", "2025-01-13", "2025-01-14", "2025-01-15", "2025-01-16", "2025-01-17"], "收盤價": [1160.0, 1130.0, 1115.0, 1110.0, 1115.0, 1090.0, 1085.0, 1115.0, 1125.0], "開盤價": [1130.0, 1105.0, 1100.0, 1100.0, 1075.0, 1090.0, 1065.0, 1105.0, 1120.0], "最高價": [1130.0, 1105.0, 1100.0, 1100.0, 1070.0, 1075.0, 1065.0, 1090.0, 1095.0], "最低價": [1130.0, 1105.0, 1100.0, 1100.0, 1075.0, 1090.0, 1065.0, 1105.0, 1120.0], "成交量": [48758688, 45211419, 27263215, 22021522, 56690300, 28695247, 35489482, 47542962, 60289321], "每日報酬": [NaN, -0.025862068965517238, -0.013274336283185861, -0.004484304932735439, 0.0045045045045044585, -0.022421524663677084, -0.004587155963302725, 0.02764976958525356, 0.008968609865470878], "調整後收盤價": [1150.0, 1110.0, 1100.0, 1100.0, 1110.0, 1085.0, 1085.0, 1095.0, 1125.0]}


### 取得股票資訊

In [ ]:
# 股票資訊查詢函數
def stock_info(stock_id = "2330.TW"):
    """
    Fetches general information about a stock.

    Args:
        stock_id (str): The stock ID (e.g., "2330.TW").

    Returns:
        dict: A dictionary containing stock information, or an error message string if the stock information can't be fetched.
            Example:{
                '公司名稱': info.get('longName', 'N/A'),
                '現價': info.get('currentPrice', 'N/A'),
                '企業簡介': info.get('longBusinessSummary', 'N/A'),
                '市值': info.get('marketCap', 'N/A'),
                '產業': info.get('industry', 'N/A'),
                '52週高點': info.get('fiftyTwoWeekHigh', 'N/A'),
                '52週低點': info.get('fiftyTwoWeekLow', 'N/A')
            }
    """
    try:
        stock = yf.Ticker(stock_id)
        info = stock.info
        return {
            '公司名稱': info.get('longName', 'N/A'),
            '現價': info.get('currentPrice', 'N/A'),
            '企業簡介': info.get('longBusinessSummary', 'N/A'),
            '市值': info.get('marketCap', 'N/A'),
            '產業': info.get('industry', 'N/A'),
            '52週高點': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52週低點': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except:
        return f"無法獲取 {stock_id} 的資訊"

In [ ]:
stock_info(stock_id = "2330.TW")

{'公司名稱': 'Taiwan Semiconductor Manufacturing Company Limited',
 '現價': 1120.0,
 '企業簡介': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support services; manufactures masks; and invests in technology start-up companies; researches, designs, develops, manufactures, packages, tests, and sells color filters; and provides investment services. Its products are used in high performance computing, smartphones, Internet of things, automotive, and digital consumer electronics. The company was incorp

### 取得基本面資料

In [ ]:
def stock_fundamental(stock_id="2330.TW"):
    """
    Fetches fundamental data for a stock (Revenue growth rate, EPS, EPS quarterly growth rate).

    Args:
        stock_id (str): The stock ID (e.g., "2330.TW").

    Returns:
       dict or None: A dictionary containing fundamental data, or None if the input is '大盤'.
          Example:{
             "季日期": dates[:len(quarterly_revenue_growth)],
             "營收成長率": quarterly_revenue_growth.tolist(),
             "EPS": quarterly_eps[:len(quarterly_revenue_growth)].tolist(),
             "EPS 季增率": quarterly_eps_growth[:len(quarterly_revenue_growth)].tolist(),
         }
        or str if an error occurs
    """
    if stock_id == "大盤":
        return None

    try:
        stock = yf.Ticker(stock_id)
    except:
        return f"無法獲取 {stock_id} 的資訊"

    # 營收成長率
    quarterly_revenue = stock.quarterly_financials.loc["Total Revenue"].dropna()
    quarterly_revenue_growth = np.round(
        quarterly_revenue.pct_change(-1, fill_method=None).dropna().tolist(), 2
    )

    # 每季 EPS
    quarterly_eps = np.round(
        stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2
    )

    # EPS 季增率
    quarterly_eps_growth = np.round(
        stock.quarterly_financials.loc["Basic EPS"].pct_change(-1, fill_method=None).dropna().tolist(), 2
    )

    # 轉換日期
    dates = [
        date.strftime("%Y-%m-%d") for date in stock.quarterly_financials.columns
    ]

    data = {
        "季日期": dates[:len(quarterly_revenue_growth)],
        "營收成長率": quarterly_revenue_growth.tolist(),
        "EPS": quarterly_eps[:len(quarterly_revenue_growth)].tolist(),
        "EPS 季增率": quarterly_eps_growth[:len(quarterly_revenue_growth)].tolist(),
    }

    return data

In [ ]:
print(stock_fundamental("2330.TW"))

{'季日期': ['2024-09-30', '2024-06-30', '2024-03-31', '2023-12-31'], '營收成長率': [0.13, 0.14, -0.05, 0.14], 'EPS': [12.54, 9.56, 8.7, 9.21], 'EPS 季增率': [0.31, 0.1, -0.06, 0.13]}


### 取得新聞資料

In [ ]:
# 新聞資料
def stock_news(stock_name="大盤"):
    """
    Fetches news articles related to a specific stock.

    Args:
        stock_name (str): The name of the stock or '大盤' to get market news.

    Returns:
        list: A list of lists, where each inner list contains [stock name, formatted_date, title, news_content]
    """
    if stock_name == "大盤":
        search = "台股 -盤中速報"
    elif "TW" in stock_name:
        search = stock_name.split(".")[0]
    else:
        search = stock_name

    data = []
    # 取得 Json 格式資料
    json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={search}&limit=5&page=1').json()

    # 依照格式擷取資料
    items = json_data['data']['items']

    for item in items:
        # 檢查 item["category"] 是否是列表，且包含至少一個元素
        if not item.get("category") or not isinstance(item["category"], list) or not item["category"]:
            continue

        # 確保 category[0] 是字典，且包含 "name" 鍵
        if not isinstance(item["category"][0], dict) or "name" not in item["category"][0]:
            continue

        if str(item["category"][0]["name"]) not in ["台股新聞", "台股公告"]:
          continue

        # 網址、標題和日期
        news_id = item["newsId"]
        title = item["title"]
        publish_at = item["publishAt"]

        # 使用 UTC 時間格式
        utc_time = dt.datetime.utcfromtimestamp(publish_at)
        formatted_date = utc_time.strftime('%Y-%m-%d')

        # 前往網址擷取內容
        url_content = requests.get(f'https://news.cnyes.com/news/id/{news_id}').content
        soup = BeautifulSoup(url_content, 'html.parser')
        p_elements = soup.find_all('p')

        # 提取段落內容
        p = ''
        for paragraph in p_elements[4:]:
            clean_text = paragraph.get_text(strip=True)
            p += clean_text

        # 移除關鍵字後的內容
        keywords = r'(按讚|訂閱|https|上一篇|下一篇|立即加入|LINE|老師)'
        p = re.split(keywords, p, maxsplit=1)[0].strip()

        # 清理重複換行
        p = re.sub(r'\n{2,}', '\n', p)  # 將多個換行合併為一個

        # 格式化數據顯示（如去掉過多小數點或雜訊）
        p = re.sub(r'(\d+\.\d{2})\d+', r'\1', p)  # 保留小數點後兩位

        # 移除一些無用的段落內容
        p = re.sub(r'(《.*?》|\=+|\-+)', '', p)  # 移除分隔線和標題
        p = p.strip()  # 清除首尾多餘的空格或換行

        # 添加整理後的內容到資料中
        data.append([stock_name, formatted_date, title, p])

    if not data:
      return f"查無{stock_name}新聞，可嘗試在'stock_news'中輸入'stock_id'"
    else:
      return data

In [ ]:
print(stock_news("台積電"))

[['台積電', '2025-01-17', '〈台股盤後〉台積電神龍擺尾 漲122點站穩2萬3 周線收紅', '今天台股大盤開盤爲 23190.12 點，最高來到 23209.88 點，最低爲 22905.53 點，，收盤報 23148.08 點，終場以上漲 122.98 點或 0.53% 作收，市場成交值爲 3203.17 億元低於 16 日的成交值，今天大盤整體集中市場電子類股上漲 0.68%，占大盤成交比重高達 73.09%。航運股上漲 0.74%，占大盤 3.68%，觀光餐旅股上漲 3.68%，占大盤 0.55%，觀光餐旅類股的大漲，主要中國文旅部今日發布，將於近期恢復福建、上海居民赴台團隊遊。中國文旅部指出，為進一步促進兩岸人員往來正常化和各領域交流常態化，將於近期恢復福建、上海居民赴台團隊遊；目前各項工作正在積極籌備中，希望兩岸旅遊業界加強溝通對接，為大陸居民赴台團隊遊提供優質的服務和產品。台積電 (2330TW) 股價今天收盤報 1120 元，上漲 15 元或 1.36%，成交量放大爲 59053 張，成交值擴增 657 億元。台積電 2024 全年稅後純益達 1 兆 2440.17 億元，年增 50.1%，每股稅後純益 45.25 元。但貨櫃航運股收盤漲跌互見，長榮 (2603TW)、萬海 (2615TW) 上漲作收，陽明 (2609TW) 收盤走跌。但長榮航 (2618TW) 在盤中以 48.3 元創新高，華航 (2610TW) 也收紅。資深證券分析師簡伯儀說，台股成交量能不足使整體交易輪動快速，且不利於中小型股，如 OTC 指數今天收跌 1.52 點或 0.61%，距離龍年封關 只剩下 3 個交易日，且春節年假長，操作仍宜慎重，目前來看，春節前的震盪將加劇，但大盤站穩 5 日線，仍對多方有利。今天收盤台股權值股台積電上漲、廣達 (2382TW) 上漲、鴻海 (2317TW) 收平盤上漲、聯發科 (2454TW) 下跌 30 元 2.05%，收盤價報 14630 元，蘋果概念股的大立光 (3008TW) 及玉晶光 (3406TW) 收盤股價下跌。昨日表現強勢的天光通訊族群，今天遭遇賣壓， 包括上詮 (3363TW) 獲外資好評，帶動族群聯亞 (3081TW) 今天仍強勢收紅。IC 載板類股走勢向上，欣興 (3037TW)、景碩 (3189TW) 

### AI Helper to generate code


In [ ]:
# 定義程式碼提取函數
def extract_code(response):
    """
    Extracts code from a string response, specifically from a markdown code block.

    Args:
        response (str): The string containing the markdown code block.

    Returns:
        str: The extracted code or 'No valid code found.' if no code block is present
            or an error message string if extraction fails.
    """
    try:
        # 使用 RegexParser 從 llm 的輸出中提取程式碼
        code_extractor = RegexParser(
            regex=r"```python\n(.*?)\n```", output_keys=["code"], default_output_key="code"
        )
        extracted_code = code_extractor.parse(response.content)
        if "code" in extracted_code and extracted_code["code"]:
            code = extracted_code["code"].strip()
            code = code.replace('```python', '').replace('```', '')
            return code
        else:
            return "No valid code found."
    except Exception as e:
        return f"Code extraction error: {str(e)}"

# 執行生成程式碼
def execute_gen_code(code_str, df):
    """
    Executes generated code on a given DataFrame.

    Args:
       code_str (str): The code to execute.
       df (pd.DataFrame): The DataFrame to process.

    Returns:
       pd.DataFrame or str: The processed DataFrame or an error message string if code execution fails.
    """
    try:
        exec(code_str, globals())
        new_df = calculate(df)
        return new_df
    except Exception as e:
        return f"Error during code execution: {str(e)}"

In [ ]:
# 定義 Prompt (用於生成程式碼)
prompt = PromptTemplate(
    input_variables=["user_msg", "columns"],
    template=(
        "You are a professional Python code generation assistant.\n"
        "Your task is to generate Python code based on specific user requirements.\n"
        "You are provided with a dataframe (df) with the columns {columns}. "
        "Generate a Python function named 'calculate(df)' that processes the dataframe "
        "according to the user requirements: {user_msg}.\n"
        "The code should:\n"
        "- Only use the given columns from the dataframe.\n"
        "- Return the processed dataframe.\n"
        "- Start with 'def calculate(df):' and only include the function definition code. Do not include any imports.\n"
        "- Enclose your code with ```python ```\n\n"
        "Strictly provide the Python code only. No explanations."
    )
)

# 建立 RunnableSequence (用於生成程式碼的 chain)
def split_input(input_dict):
    input_str = input_dict["input"]
    user_msg, columns = input_str.split(", columns: ", 1)
    return {"user_msg": user_msg, "columns": columns}

code_gen_chain = RunnableSequence(
    RunnablePassthrough() |
    RunnableLambda(split_input) |
    prompt |
    llm
)

In [ ]:
# ai_helper 函數 (使用 LangChain)
def ai_helper(input_str):
    """
    Generates Python code using an LLM based on user message and DataFrame columns loaded from a CSV file,
    and then executes it to update the CSV.

    After generating the code, you can use 'extract_stock_price' tool to get stock price information.

    Args:
        stock_price_path (str): The path to the CSV file containing stock price data,
                            which should contain columns that the generated code will use.
        user_msg (str): The user's request for code generation.

    Returns:
        str: If successful, return a message containing the new stock_price_path,
             example: "New data is created. {\"stock_price_path\" : \"{stock_price_path}\"}"
             or an error message if code generation or execution fails.

    Raises:
        FileNotFoundError: If the provided file path does not exist.
        Exception: If any other error occurs during the process.
    """
    try:
        stock_price_path, user_msg = input_str.split(",")
        df = pd.read_csv(stock_price_path)
        df_columns = df.columns.tolist()
        input_data = {"input": f"{user_msg}, columns: {', '.join(df_columns)}"}
        response = code_gen_chain.invoke(input_data)
        code_str = extract_code(response)
        df = execute_gen_code(code_str,df)
        df.to_csv(stock_price_path, index=False, encoding='utf-8-sig', float_format="%.2f")
        return f"New data is created. \"stock_price_path\" : \"{stock_price_path}\""
    except Exception as e:
        return f"Error in ai_helper: {str(e)}"

In [ ]:
stock_price_path = "stock_price/2330.TW.csv"
user_msg = "MACD"

text = ai_helper(f"{stock_price_path},{user_msg}")
print(text)

New data is created. "stock_price_path" : "stock_price/2330.TW.csv"


In [ ]:
df = pd.read_csv("stock_price/2330.TW.csv")
df

,Date,調整後收盤價,收盤價,最高價,最低價,開盤價,成交量
0,2025-01-07,1150.0,1160.0,1130.0,1130.0,1130.0,48758688
1,2025-01-08,1110.0,1130.0,1105.0,1105.0,1105.0,45211419
2,2025-01-09,1100.0,1115.0,1100.0,1100.0,1100.0,27263215
3,2025-01-10,1100.0,1110.0,1100.0,1100.0,1100.0,22021522
4,2025-01-13,1110.0,1115.0,1070.0,1075.0,1075.0,56690300
5,2025-01-14,1085.0,1090.0,1075.0,1090.0,1090.0,28695247
6,2025-01-15,1085.0,1085.0,1065.0,1065.0,1065.0,35489482
7,2025-01-16,1095.0,1115.0,1090.0,1105.0,1105.0,47542962
8,2025-01-17,1125.0,1125.0,1095.0,1120.0,1120.0,60289321


### LSTM預測模型

In [ ]:
!rm -R /content/Stock_Analysis
!git clone https://github.com/steak0069/Stock_Analysis/

rm: cannot remove '/content/Stock_Analysis': No such file or directory
Cloning into 'Stock_Analysis'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 29.68 MiB | 11.86 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

model_path = "/content/Stock_Analysis/predictions_model/trained_model.keras"

# 載入儲存的模型
loaded_model = load_model(model_path)

In [ ]:
def predicted_stock_price(stock_price_path):
    """
    Predicts the next day's stock price using a pre-trained LSTM model.

    Args:
        stock_price_path (str): The path to the CSV file containing historical stock price data,
                                 which should contain a '收盤價' column.

    Returns:
        str: A string containing the predicted next day's stock price, formatted to two decimal places.
              Example: "預測的明天股價：123.45"
           or str if an error occurs

    Raises:
        FileNotFoundError: If the provided file path does not exist.
        KeyError: If the CSV file does not contain a '收盤價' column.
        Exception: If any other error occurs during the prediction process.
    """
    try:
        df = pd.read_csv(stock_price_path)
        data = df['收盤價']

        # 將 Series 轉換為 2 維陣列
        data_reshaped = data.values.reshape(-1, 1)

        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data_reshaped)

        Xlstm = scaled_data[-60:]
        Xlstm = Xlstm.reshape(1, Xlstm.shape[0], 1)   # Xlstm.shape = (1, 60, 1)

        # 假設 loaded_model 已經被定義
        predicted_price = loaded_model.predict(Xlstm, verbose=0)
        predicted_price = scaler.inverse_transform(predicted_price)

        return f"預測的明天股價：{predicted_price[0][0]:.2f}"

    except FileNotFoundError:
      return f"[Error] No such file or directory: {stock_price_path}"
    except KeyError:
        return f"[Error] CSV file does not contain a '收盤價' column."
    except Exception as e:
        return f"[Error] An error occurred: {str(e)}"

In [ ]:
predicted_stock_price("stock_price/2330.TW.csv")

'預測的明天股價：1109.27'

# RAG

### 加載向量資料

In [ ]:
vector_db_path = "/content/Stock_Analysis/vectordb/stockvector.db"

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3",
encode_kwargs={"normalize_embeddings": True})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# 加載向量資料
vectordb = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
# Create retriever for later use
retriever = vectordb.as_retriever()

### Retrieval_chain



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
template = """
You are a Q&A chat bot.
Use the given context only, answer the question.

<context>
{context}
</context>

Question: {input}
"""

# Create a prompt template
prompt = ChatPromptTemplate.from_template(template)
doc_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, doc_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "什麼是黃金交叉?"})

print(response['answer'])

根據提供的內容，沒有明確的定義或解釋什麼是“黃金交叉”。然而，根據圖2.3.3中的GBPUSD日線圖，圖中顯示了移動平均線（MA 50 & MA 200），這可能與黃金交叉相關。黃金交叉通常是指短期移動平均線（例如MA 50）上穿長期移動平均線（例如MA 200）的情況，表明市場趨勢可能從下降轉為上升。然而，這個解釋並不是直接從提供的內容中得出的。


# Agent

In [ ]:
# 主要 Agent 的 Prompt 和初始化
main_agent_prompt = PromptTemplate(
    input_variables=["user_question", "knowledge"],
    template=(
        "You are a financial analyst agent. Your task is to analyze the user's question about the stock market,you can explain the data in a professional and understandable way.\n"
        "You should follow the following keys:\n"
        "- `user_question`: The original user question.\n"
        "- `knowledge`: Relevant knowledge retrieved from the vector database, if available.\n"
        "- `stock_id`: The stock ID (e.g., '2330.TW') if the question is about a specific stock. Otherwise, leave it as an empty string.\n"
        "- `stock_name`: The stock name (e.g., '台積電') if the question is about a specific stock. Otherwise, leave it as an empty string.\n"
        "- `days`: The number of days of stock data needed. If not explicitly stated in the question, or not inferable, set it to 30.\n"
        "- `stock_price_path`: The path to the CSV file with stock price data, if the 'stock_price' tool is called, otherwise leave it as an empty string\n"
        "- `stock_data`: A dictionary containing data fetched from the tools:\n"
        "    - `stock_price`: Stock price data if needed, otherwise an empty dictionary.\n"
        "    - `stock_info`: Stock information if needed, otherwise an empty dictionary.\n"
        "    - `stock_fundamental`: Fundamental stock data if needed, otherwise an empty dictionary.\n"
        "    - `stock_news`: News about the stock if needed, otherwise an empty string.\n"
        "- `code_str`: The code generated by ai_helper if needed, otherwise an empty string.\n"
        "Your process should follow these steps:\n"
        "1. Analyze the `user_question` and the `knowledge` to understand the user's needs.\n"
        "2. Use tools to retrieve the required stock data. Always use 'query_stock' to get stock_id first if user question related to a specific stock.\n"
        "3. If the user question explicitly asks for a 'prediction' or 'forecast', and you have 60 days stock price data and the stock price path, then use `predicted_stock_price` tool to predict the next day's stock price. It returns a string of prediction, which should be included in the final output. Do not use `predicted_stock_price` tool in other cases.\n"
        "4. If user need the technical indicators data such as 'MA'、'MACD'、'RSI'... first you need to perpare 60 days stock price data from 'stock_price', then call the `ai_helper` tool to generate the necessary data, finally use 'extract_stock_price' to read them.\n"
        "5. Summarize all the information and put it in analysis, final response your analysis and the answer\n"
        "6. If the user question does not refer to any specific stocks, leave `stock_id` and `stock_name` as empty string, `days` can be set as default 30 days, and do not use the stock data related tools. Only retrieve relevant information from the `knowledge` and create the final response.\n"
        "User question: {user_question}\n"
        "knowledge: {knowledge}\n"
    )
)


# 主要 Agent 工具列表
main_agent_tools = [
    Tool(
        name="query_stock",
        func=query_stock,
        description="Queries stock info by name or ID. Returns: 'stock_name', 'stock_id'."
    ),
    Tool(
        name="stock_price",
        func=stock_price,
        description="Fetches historical stock price data. Input: stock ID (str), days (int). Returns: 'stock_price_path'. Use 'extract_stock_price' for price details."
    ),
    Tool(
        name="stock_info",
        func=stock_info,
        description="Fetches stock info. Input: stock ID (str). Returns: 'company_name', 'current_price', 'business_summary', 'market_cap', 'industry', '52_week_high', '52_week_low'"
    ),
    Tool(
       name = "stock_fundamental",
       func= stock_fundamental,
       description= "Fetches fundamental data. Input: stock ID (str). Returns: 'quarterly_dates', 'revenue_growth_rate', 'EPS', 'EPS_quarterly_growth_rate' or None if 大盤."
    ),
    Tool(
       name = "stock_news",
       func= stock_news,
       description= "Fetches news. Input: stock name (str) or '大盤'. Returns: list of news."
    ),
     Tool(
        name="ai_helper",
        func=ai_helper,
        description="Generates and executes Python code on CSV data based on user request. Input: CSV path (str), user message (str). Returns: message with 'stock_price_path'. Then use 'extract_stock_price' for price details."
    ),
    Tool(
        name="extract_stock_price",
        func=extract_stock_price,
         description="Extracts stock price details, only the latest 20 records. Input: CSV path (str). Returns: 'dates', 'closing_prices', 'opening_prices', 'highest_prices', 'lowest_prices', 'volume', 'daily_returns', 'price_differences', and other 'ai_helper' generate."
    ),
    Tool(
        name="predicted_stock_price",
        func=predicted_stock_price,
        description="Use only if the user asks for a 'prediction' or 'forecast'. Predicts the next day's stock price. Input: stock's CSV path (str). Returns: string with predicted price."
    )
]

# 初始化 Agent
main_agent = initialize_agent(
    tools=main_agent_tools,
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=False,
    input_variables=["user_question"],
)

In [ ]:
def stock_agent(user_question):
  # 1. 執行檢索 Chain
  retrieval_output = retrieval_chain.invoke({"input": user_question})

  # 2. 主要 Agent 執行
  main_agent_output = main_agent.invoke({
      "input": user_question,
      "knowledge": str(retrieval_output['answer'])
  })

  return main_agent_output['output']


In [ ]:
stock_agent("台積電是做什麼的")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "query_stock",
  "action_input": "台積電"
}
```

Observation: {'股名': '台積電', '股號': '2330.TW'}
Thought:Now that we have the stock ID, let's get more information about the company.

Action:
```
{
  "action": "stock_info",
  "action_input": "2330.TW"
}
```


Observation: {'公司名稱': 'Taiwan Semiconductor Manufacturing Company Limited', '現價': 1120.0, '企業簡介': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support services; manufactures

'台積電是一家半導體製造公司，主要業務包括晶圓製造、封裝、測試和銷售積體電路和其他半導體元件。其產品廣泛應用於高性能計算、智能手機、物聯網、汽車和數字消費電子等領域。'

# 生成技術分析圖

## 連接OpenAI模型

In [ ]:
from  openai import OpenAI, OpenAIError # 串接 OpenAI API
import plotly.graph_objects as go

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# GPT 3.5 模型
def get_reply(messages):
  try:
    response = client.chat.completions.create(model="gpt-3.5-turbo",
                         messages=messages)
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"發生 {err.type} 錯誤\n{err.message}"
  return reply

# 設定 AI 角色, 使其依據使用者需求進行 df 處理
def openai_helper(df, user_msg):

  msg = [{
    "role":
    "system",
    "content":
    f"As a professional code generation robot, \
      I require your assistance in generating Python code \
      based on specific user requirements. To proceed, \
      I will provide you with a dataframe (df) that follows the \
      format {df.columns}. Your task is to carefully analyze the \
      user's requirements and generate the Python code \
      accordingly.Please note that your response should solely \
      consist of the code itself, \
      and no additional information should be included."
  }, {
    "role":
    "user",
    "content":
    f"The user requirement:{user_msg} \n\
       Your task is to develop a Python function named \
      'calculate(df)'. This function should accept a dataframe as \
      its parameter. Ensure that you only utilize the columns \
      present in the dataset, specifically {df.columns}. \
      After processing, the function should return the processed \
      dataframe. Your response should strictly contain the Python \
      code for the 'calculate(df)' function \
      and exclude any unrelated content."
  }]

  reply_data = get_reply(msg)
  cleaned_code = reply_data.replace("```", "")
  cleaned_code = cleaned_code.replace("python", "")

  return cleaned_code

## 繪製圖片

In [ ]:
import plotly
import kaleido

In [ ]:
image_dir = 'stock_technical_chart'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

In [ ]:
from PIL import Image

temp_path = f"{image_dir}/temp.png"
original_image_path = f"{image_dir}/stock_chart.jpg"  # 請替換為你的圖片路徑
preview_image_path = f"{image_dir}/stock_chart_preview.jpg"  # 輸出的圖片檔案名

# 設定目標尺寸
original_image_size = (1200, 800)
preview_image_size = (240, 240)

def size_change():
  # 開啟圖片並調整大小
  try:
      with Image.open(temp_path) as img:
        # 如果圖片有透明度，先轉換為 RGB 模式
        if img.mode in ("RGBA", "P"):  # 檢查是否帶透明通道
            img = img.convert("RGB")

        # 調整大小
        original_image = img.resize(original_image_size, Image.Resampling.LANCZOS)
        preview_image = img.resize(preview_image_size, Image.Resampling.LANCZOS)

        # 儲存圖片為 JPG 格式
        original_image.save(original_image_path, format="JPEG")
        preview_image.save(preview_image_path, format="JPEG")
  except Exception as e:
      print(f"處理圖片時發生錯誤：{e}")

In [ ]:
# 下載資料並讓 AI 計算指標
def download_stock_data(stock_id, start=None, end=None, indicator='MACD'):
    stock_id = f"{stock_id}.tw"
    if not end:
        end = dt.date.today()
    if not start:
        start = end - dt.timedelta(days=365)
    # 從 yf 下載資料
    df = yf.download(stock_id, start=start, end=end).reset_index()

    # AI 計算技術指標
    code_str = openai_helper(df, f"計算{indicator}")
    print(code_str)

    # 將 exec 生成的 calculate 設為局部變數
    local_vars = {}
    exec(code_str, globals(), local_vars)
    calculate = local_vars['calculate']

    df = calculate(df)

    # 資料處理
    bk_df = df.reset_index()
    bk_df.index = bk_df["Date"].dt.strftime('%Y-%m-%d')

    return bk_df

# 繪製圖表函式
def create_stock_figure(stock_id, bk_df):

    # 創建 K 線圖
    fig = go.Figure(data=[go.Candlestick(x=bk_df.index,
                        open=bk_df['Open'],
                        high=bk_df['High'],
                        low=bk_df['Low'],
                        close=bk_df['Close'],
                        increasing_line_color='red',
                        decreasing_line_color='green',
                        name = "K 線")])

    # 交易量
    fig.add_trace(go.Bar(x=bk_df.index, y=bk_df['Volume'],
                         marker={'color': 'green'}, yaxis='y2',
                           name = "交易量"))

    # 找出需要繪製的欄位
    columns = bk_df.columns
    exclude_columns = ['index','Date', 'Open', 'High',
                        'Low', 'Close', 'Adj Close', 'Volume']
    remain_columns = [col for col in columns if
                       col not in exclude_columns]
    min_close = bk_df['Close'].min() - bk_df['Close'].std()
    max_close = bk_df['Close'].max() + bk_df['Close'].std()
    # 繪製技術指標
    for i in remain_columns:
      if min_close <= bk_df[i].mean() <= max_close:
        fig.add_trace(go.Scatter(x=bk_df.index, y=bk_df[i],
                                  mode='lines', name=i))
      else:
        fig.add_trace(go.Scatter(x=bk_df.index, y=bk_df[i],
                                  mode='lines', yaxis='y3', name=i))

    # 加入懸停十字軸
    fig.update_xaxes(showspikes=True, spikecolor="gray",
                    spikemode="toaxis")
    fig.update_yaxes(showspikes=True, spikecolor="gray",
                    spikemode="across")
    # 更新畫布大小並增加範圍選擇
    fig.update_layout(
        height=800,
        width=1200,
        yaxis={'domain': [0.35, 1]},
        yaxis2={'domain': [0.15, 0.3]},
        # 若要重疊 y1 和 y3, 可以改成
        # yaxis3=dict(overlaying='y', side='right')
        yaxis3={'domain': [0, 0.15]},
        title=f"{stock_id}",
        xaxis={
            # 範圍選擇格
            'rangeselector': {
                'buttons': [
                    {'count': 1, 'label': '1M',
                      'step': 'month', 'stepmode': 'backward'},
                    {'count': 6, 'label': '6M',
                      'step': 'month', 'stepmode': 'backward'},
                    {'count': 1, 'label': '1Y',
                      'step': 'year', 'stepmode': 'backward'},
                    {'step': 'all'}
                ]
            },
            # 範圍滑動條
            'rangeslider': {
                'visible': True,
                'thickness': 0.01,  # 滑動條的高度
                'bgcolor': "#E4E4E4"  # 背景色
            },
            'type': 'date'
        }
    )

    # 移除非交易日空值
    # 生成該日期範圍內的所有日期
    all_dates = pd.date_range(start=bk_df.index.min(),
                               end=bk_df.index.max())
    # 找出不在資料中的日期
    breaks = all_dates[~all_dates.isin(bk_df.index)]
    dt_breaks = breaks.tolist() # 轉換成列表格式
    fig.update_xaxes(rangebreaks=[{'values': dt_breaks}])
    return fig

# 主函式
def plotly_stock(stock_id, start=None, end=None, indicator='MACD'):
    df = download_stock_data(stock_id, start, end, indicator)
    fig = create_stock_figure(stock_id,df)
    fig.write_image(temp_path)
    size_change()
    fig.show()

In [ ]:
plotly_stock("2330", start='2022-01-01', end= None, indicator='布林通道及MACD')

[*********************100%%**********************]  1 of 1 completed


def calculate(df):
    # Calculate Bollinger Bands
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['20 Day STD'] = df['Close'].rolling(window=20).std()
    df['Upper Band'] = df['MA20'] + (df['20 Day STD'] * 2)
    df['Lower Band'] = df['MA20'] - (df['20 Day STD'] * 2)
    
    # Calculate MACD
    df['26 Day EMA'] = df['Close'].ewm(span=26).mean()
    df['12 Day EMA'] = df['Close'].ewm(span=12).mean()
    df['MACD'] = df['12 Day EMA'] - df['26 Day EMA']
    
    return df


<ipython-input-137-c0ab5b497df5>:106: FutureWarning:

The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.



# LINE Bot

### Server

In [ ]:
from google.colab import userdata
# LINE Developers
LINE_TOKEN = userdata.get('LINE_TOKEN')
LINE_SECRET = userdata.get('LINE_SECRET')
USER_ID = userdata.get('USER_ID')

# ngrok
ngrok_authtoken = userdata.get('ngrok_authtoken')

In [ ]:
from flask import Flask, request, abort, send_from_directory
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent,
    TextMessage,
    TextSendMessage,
    ImageSendMessage
)
from pyngrok import ngrok


# 設置 ngrok authtoken
ngrok.set_auth_token(ngrok_authtoken)

# 初始化 Line Bot API 和 WebhookHandler
api = LineBotApi(LINE_TOKEN)
handler = WebhookHandler(LINE_SECRET)

app = Flask(__name__)

# 靜態圖片目錄（用於提供圖片資源）
STATIC_IMAGE_FOLDER = image_dir
os.makedirs(STATIC_IMAGE_FOLDER, exist_ok=True)


@app.route("/", methods=["POST"])
def callback():
    """
    Webhook 回呼函式，用於接收來自 Line 的訊息。
    """
    # 取得 X-Line-Signature 表頭電子簽章內容
    signature = request.headers.get('X-Line-Signature')

    # 以文字形式取得請求內容
    body = request.get_data(as_text=True)
    app.logger.info(f"Request body: {body}")

    # 驗證電子簽章並處理請求內容
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.error("電子簽章錯誤，請檢查密鑰是否正確")
        abort(400)

    return 'OK'


@app.route(f"/{image_dir}/<filename>")
def serve_image(filename):
    """
    提供靜態圖片資源。
    """
    return send_from_directory(STATIC_IMAGE_FOLDER, filename)


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    """
    處理來自用戶的訊息事件。
    """
    user_message = event.message.text.strip()

    try:
        if "圖表" in user_message:
            # 移除 "圖表"
            user_message = user_message.replace("圖表", "").strip()

            # 使用空格分開股票代號和技術指標
            parts = user_message.split(" ", 1)  # 只分割第一次出現的空格

            if len(parts) == 2:
                stock_number = parts[0]  # 股票代號
                indicator = parts[1]   # 技術指標

                #傳送股票代碼
                plotly_stock(stock_number, start='2022-01-01', end= None,
                              indicator=indicator)   #生成圖片
                # 圖片的公開 URL
                image_url = f"{ngrok_url}/{original_image_path}"

                # 回應一張圖片
                image_message = ImageSendMessage(
                    original_content_url=
                      f"{ngrok_url}/{original_image_path}",  # 圖片完整尺寸 URL
                    preview_image_url=
                      f"{ngrok_url}/{preview_image_path}"  # 預覽圖片 URL
                )
                api.reply_message(event.reply_token, image_message)
            else:
                reply_text = "格式錯誤，正確格式:2330(空格)13MA圖表"
                api.reply_message(
                    event.reply_token,
                    TextSendMessage(text=reply_text)
                )

        else:   # 進入AI agent
            reply_text = stock_agent(user_message)
            api.reply_message(
                event.reply_token,
                TextSendMessage(text=reply_text)
            )
    except Exception as e:
        app.logger.error(f"處理訊息時發生錯誤：{e}")
        api.reply_message(
            event.reply_token,
            TextSendMessage(text="很抱歉，處理您的訊息時發生錯誤，請稍後再試。")
        )


if __name__ == "__main__":
    # 啟動 ngrok
    public_url = ngrok.connect(5000)
    ngrok_url = public_url.public_url  # 取得公開的 ngrok URL
    app.logger.info(f"ngrok URL: {ngrok_url}")

    # 將 ngrok URL 提供給開發者
    print(f"公開 URL：{ngrok_url}\n")

    # 啟動 Flask 應用
    app.run(host="0.0.0.0", port=5000)

<ipython-input-183-72043365fb80>:17: LineBotSdkDeprecatedIn30:

Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.

<ipython-input-183-72043365fb80>:18: LineBotSdkDeprecatedIn30:

Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.



公開 URL：https://7e7b-34-80-245-12.ngrok-free.app

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 10:20:41] "POST / HTTP/1.1" 200 -


給我鴻海的股票資訊


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "query_stock",
  "action_input": "鴻海"
}
```

Observation: {'股名': '鴻海', '股號': '2317.TW'}
Thought:Action:
```
{
  "action": "stock_info",
  "action_input": "2317.TW"
}
```
Observation: {'公司名稱': 'Hon Hai Precision Industry Co., Ltd.', '現價': 173.5, '企業簡介': 'Hon Hai Precision Industry Co., Ltd. provides electronic OEM services. The company offers consumer electronics comprising of television sets, game consoles, set-top boxes, and speakers. It also provides cloud networking products consisting of routers, servers, edge computing, data centers, and satellite communications and other related equipment. In addition, the company offers desktop computers, notebook computers, tablets, business machines, printers, etc. Further, it provides connectors, precision optical components, lenses, electronic components, semiconductor products, automotive electronic parts, cutting tools/mold fixtures, and mechanical equipment. The c

<ipython-input-183-72043365fb80>:101: LineBotSdkDeprecatedIn30:

Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.

INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 10:21:03] "POST / HTTP/1.1" 200 -


...

什麼是黃金交叉


> Entering new AgentExecutor chain...


<ipython-input-183-72043365fb80>:101: LineBotSdkDeprecatedIn30:

Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.

INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 10:24:24] "POST / HTTP/1.1" 200 -


Thought: 黃金交叉是一種技術分析指標，指短期移動平均線（通常為50天）上穿長期移動平均線（通常為200天），被視為牛市的信號。

Action:
```
{
  "action": "Final Answer",
  "action_input": "黃金交叉是一種技術分析指標，指短期移動平均線（通常為50天）上穿長期移動平均線（通常為200天），被視為牛市的信號。"
}
```

> Finished chain.
什麼是黃金交叉
台積電股價


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "query_stock",
  "action_input": "台積電"
}
```

Observation: {'股名': '台積電', '股號': '2330.TW'}
Thought:Action:
```
{
  "action": "stock_price",
  "action_input": "2330.TW, 30"
}
```

Observation: {'stock_price_path': 'stock_price/2330.TW.csv'}
Thought:Action:
```
{
  "action": "extract_stock_price",
  "action_input": "stock_price/2330.TW.csv"
}
```

Observation: {'日期': ['2024-12-20', '2024-12-23', '2024-12-24', '2024-12-25', '2024-12-26', '2024-12-27', '2024-12-30', '2024-12-31', '2025-01-02', '2025-01-03', '2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-13', '2025-01-14', '2025-01-15', '2025-01-16', '2025-01-17'], '收盤價': [1060.0, 1080.0, 1095.0, 1090.0, 1090.0, 1095.0, 

<ipython-input-183-72043365fb80>:101: LineBotSdkDeprecatedIn30:

Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.

INFO:werkzeug:127.0.0.1 - - [17/Jan/2025 10:25:03] "POST / HTTP/1.1" 200 -


Action:
```
{
  "action": "Final Answer",
  "action_input": "台積電的最新股價是1125.0"
}
```

> Finished chain.
台積電股價
